In [15]:
import data_gen as gen
from typing import List
from math import sqrt, log, pow, inf

In [16]:
budget = 50000
website_list = ["111","112","113","122","123"]
data = gen.generate_data(budget, website_list, 0.1, 0.3, 0.01)
data


{'111': <data_gen.Website at 0x107921790>,
 '112': <data_gen.Website at 0x1181caa00>,
 '113': <data_gen.Website at 0x107532b20>,
 '122': <data_gen.Website at 0x107532850>,
 '123': <data_gen.Website at 0x107532af0>}

In [80]:
arms = [data[arm_name] for arm_name in data.keys()]
print(sum(arms[0].realisations))
print(sum(arms[1].realisations))
print(sum(arms[2].realisations))
print(sum(arms[3].realisations))
print(sum(arms[4].realisations))

31305
31763
31239
33619
32422


In [45]:
def calc_beta(arm, a):
    return sqrt(a/arm.num)

In [46]:
def get_max_key(input_dict,keys):
    new_dict = dict()
    for item in keys:
        new_dict[item] = input_dict[item]
    return max(new_dict, key=new_dict.get)


def get_min_key(input_dict, keys):
    new_dict = dict()
    for item in keys:
        new_dict[item] = input_dict[item]
    return min(new_dict, key=new_dict.get)


In [14]:
def select_arm(current_arms, m, a):   
    current_arms = [data[arm_name] for arm_name in data.keys()]

    ukt_dict = dict()
    lkt_dict = dict()
    bkt_dict = dict()
    beta_dict = dict()
    # Compute Uk(t) and Lk(t) for each arm\n",
    for k in current_arms:
        beta_dict[k] = calc_beta(k, a)
        ukt = k.last_average + beta_dict[k]
        lkt = k.last_average - beta_dict[k]
        ukt_dict[k] = ukt
        lkt_dict[k] = lkt
    # Compute Bk(t) for each arm\n",
    for k in current_arms:
        ukt_dict_subset = ukt_dict.copy()
        ukt_dict_subset.pop(k)
        max_ukt = max(ukt_dict_subset, key=ukt_dict_subset.get)
        bkt = ukt_dict[max_ukt] - lkt_dict[k]
        bkt_dict[k] = bkt

    # Identify the set of m arms J(t)\n",
    bkt_dict_sorted = dict(sorted(bkt_dict.items(), key=lambda item: item[1]))
    Jt = set(list(bkt_dict_sorted.keys())[:m])

    # Identify arm with minimum lower bound in J(t)\n",
    lowest = inf
    contenders = []
    for arm in Jt:
        if lkt_dict[arm] <= lowest:
            contenders.append(arm)
    if len(contenders) > 1:
        lower = get_min_key(beta_dict, contenders)
    else:
        lower = contenders[0]
    # Identify arm with maximum upper bound *not* in J(t)\n",
    not_in_jt = list(set(ukt_dict.keys()) - Jt)
    highest = -inf
    contenders = []
    for arm in not_in_jt:
        if ukt_dict[arm] >= highest:
            contenders.append(arm)
    if len(contenders) > 1:
        upper = get_max_key(beta_dict, contenders)
    else:
        upper = contenders[0]
    # Identify and pull arm I(t)
    keyz = [lower, upper]
    It = get_max_key(beta_dict, keyz)
    It.pull()

    return Jt, bkt_dict


In [81]:
def UGapEb(data, m, budget, a):
    K = len(data)
    ## Initialise
    for arm_name in data.keys():
        data[arm_name].pull()
    ## Main loop
    Jt = ''
    bkt_dict = ''
    t = 0
    for t in range(K+1,budget):
        
        current_arms = [data[arm_name] for arm_name in data.keys()]
        Jt, bkt_dict = select_arm(current_arms, m, a)

    return get_min_key(bkt_dict, Jt)
    
print(UGapEb(data, 3, 5000, 0.1).name)

for website in data.keys():
    print(
        f'website: {data[website].name}, num: {data[website].num}, average: {data[website].average}')


122
website: 111, num: 9, average: 0.7777777777777778
website: 112, num: 1663, average: 0.6386049308478653
website: 113, num: 1663, average: 0.6193625977149729
website: 122, num: 2, average: 1.0
website: 123, num: 1662, average: 0.6377858002406739


In [75]:
def UGapEc(data, e, m, delta, c):
    K = len(data)
    
    ## Initialise arms
    for arm_name in data.keys():
        data[arm_name].pull()

    ## Main loop
    Jt = ''
    Bkt_dict = ''
    t = K + 1
    stopping_criteria = True
    while (stopping_criteria):
        current_arms = [data[arm_name] for arm_name in data.keys()]

        a = c * log((4*K*(t**3))/delta)
        Jt, Bkt_dict = select_arm(current_arms, m, a)

        key = get_max_key(Bkt_dict, Jt)
        t += 1
        if (Bkt_dict[key] >= e):
            stopping_criteria = False
        
    return Jt

In [89]:
delta = 0.9
c = 0.5
optimal_arms = UGapEc(data, 0.6, 3, delta, c)
for arm in optimal_arms:
    print(arm.name)

112
123
113


TypeError: 'set' object is not subscriptable